In [1]:
import sys
sys.path.insert(0,"C:\\Program Files\\slab-master\\")
sys.path.insert(1,"C:\\Users\\MeasPC\\")

In [2]:
from slab.instruments import PNAX
from slab import SlabFile

No module named 'visa'
Warning VISA library import failed
Warning serial library import failed.
Could not load Autonics TM4 Driver


In [3]:
pna=PNAX.N5242A(address="169.254.109.0")

TimeoutError: [Errno 60] Operation timed out

In [ ]:
pna.get_id()

In [ ]:
import pnax_spectroscopy_py3

In [ ]:
import os
os.getcwd()

In [ ]:
from slab import *
from slab.datamanagement import SlabFile
from numpy import *
import os
import time
from slab.instruments import *
from slab.instruments import InstrumentManager

im = InstrumentManager()
nwa=PNAX.N5242A(address="169.254.109.0")
#nwa =  im['PNAX2']
print(nwa.get_id())
print('Deviced Connected')

expt_path = os.getcwd() + '\data'

# initial NWA configuration values
ifbw = 1e3
read_power = -55
probe_power = -10
sweep_pts = 1000
avgs = 100
avgs_state = 1
delay = -12.41e-12

prefix = "PNAX_%s_flute cavity"

print("Configuring the NWA")

print ("NWA Configured. IFBW = %f Hz, Read_Power = %f, Drive_Power = %f, NumPts = %d, Avgs = %d " %(ifbw,read_power,probe_power,sweep_pts,avgs))

def single_tone_CW(read_freq_center,span,read_power = read_power,probe_power=probe_power,ifbw = ifbw, sweep_pts= sweep_pts,avgs=avgs,
                   avgs_state=avgs_state,is_qubitdrive= 0,qfreq= 0,measurement="S21"):

    nwa.set_timeout(10e5)
    nwa.set_ifbw(ifbw)
    nwa.set_sweep_points(sweep_pts)
    nwa.clear_traces()
    nwa.setup_measurement(measurement)
    nwa.set_electrical_delay(delay, channel=1)

    vary_param = "single_tone_CW"
    print("Swept Parameter: %s" % (vary_param))
    prefix = "Readout_%s" % vary_param
    fname = get_next_filename(expt_path, prefix, suffix='.h5')
    print(fname)
    fname = os.path.join(expt_path, fname)

    nwa.setup_take(averages=avgs)
    nwa.set_average_state(avgs_state)

    read_freq_start = read_freq_center - span/2.0
    read_freq_stop =  read_freq_center + span/2.0

    print ("Setting up CW parameters")

    nwa.set_ifbw(ifbw)

    nwa.write('SENSE:FOM:STATE 1')
    nwa.write("sense:fom:range2:coupled 1")
    nwa.write("sense:fom:range3:coupled 1")
    nwa.write("sense:fom:range4:coupled 0")
    nwa.write('SENSE:FOM:RANGE1:FREQUENCY:START %f' % read_freq_start)
    nwa.write('SENSE:FOM:RANGE1:FREQUENCY:STOP %f' % read_freq_stop)

    nwa.write('SENSE:FOM:RANGE4:FREQUENCY:START %f' % qfreq)
    nwa.write('SENSE:FOM:RANGE4:FREQUENCY:STOP %f' % qfreq)
    
    nwa.write(":SOURCE:POWER1 %f" % (read_power))
    nwa.write(":SOURCE1:POWER1:MODE ON")
    print ("Read power",read_power)
    print ("Read Now On")


    #setting the port powers and decoupling the powers
    if measurement == 'S21':
        nwa.write(":SOURCE:POWER3 %f" % (probe_power))
    if is_qubitdrive:
        nwa.write(":SOURCE1:POWER3:MODE ON")
        print ("Qubit Drive Now ON")
    else:
        nwa.write(":SOURCE1:POWER3:MODE OFF")
        print ("Qubit Drive Now Off")

    data = nwa.take_one_in_mag_phase()
    fpoints = data[0]
    mags = data[1]
    phases = data[2]
    print ("finished downloading")
    
    with SlabFile(fname) as f:
        f.append_line('mags', mags)
        f.append_line('phases', phases)
        f.append_line('freq', fpoints)
        f.append_pt('read_power', read_power)
        f.append_pt('probe_power', probe_power)

    print (fname)

def single_tone_power_sweep(read_power_list,read_freq_center,span,ifbw = ifbw, sweep_pts= sweep_pts,avgs=avgs,avgs_state=avgs_state,is_qubitdrive= 0,qfreq= 0,measurement="S21"):

    nwa.set_timeout(10E5)
    nwa.set_ifbw(ifbw)
    nwa.set_sweep_points(sweep_pts)
    nwa.clear_traces()
    nwa.setup_measurement(measurement)
    nwa.set_electrical_delay(delay, channel=1)


    vary_param = "single_tone_CW"

    prefix = "Single_tone_nearReadoutFreq_Transmission"
    fname = get_next_filename(expt_path, prefix, suffix='.h5')
    print(fname)
    fname = os.path.join(expt_path, fname)



    #setting the port powers and decoupling the powers

    for read_power in read_power_list:
        nwa.setup_take(averages=avgs)
        nwa.set_average_state(avgs_state)

        if read_power < -30:
            nwa.setup_take(averages=5)
        elif read_power < -25:
            nwa.setup_take(averages=5)
        else:
            nwa.setup_take(averages=1)



        read_freq_start = read_freq_center - span / 2.0
        read_freq_stop = read_freq_center + span / 2.0

        print("Setting up CW parameters")

        nwa.write('SENSE:FOM:STATE 1')
        nwa.write("sense:fom:range2:coupled 1")
        nwa.write("sense:fom:range3:coupled 1")
        nwa.write("sense:fom:range4:coupled 0")
        nwa.write('SENSE:FOM:RANGE1:FREQUENCY:START %f' % read_freq_start)
        nwa.write('SENSE:FOM:RANGE1:FREQUENCY:STOP %f' % read_freq_stop)

        nwa.write('SENSE:FOM:RANGE4:FREQUENCY:START %f' % qfreq)
        nwa.write('SENSE:FOM:RANGE4:FREQUENCY:STOP %f' % qfreq)

        # Turning off pulsed aspect of the measurement (just in case it was run before)
        # Turning the leveling of the ports to 'Internal'.   'open-loop' is for pulsed
        nwa.write("source1:power1:alc:mode INTERNAL")
        nwa.write("source1:power3:alc:mode INTERNAL")
        # Setting up the proper trigger mode.   Want to trigger on the point
        nwa.write("TRIG:SOUR EXT")
        nwa.write("SENS:SWE:TRIG:MODE POINT")

        # Turning off the "Reduce IF BW at Low Frequencies" because the PNA-X adjusts the BW automatically to correct for roll-off at low frequencies
        nwa.write("SENS:BWID:TRAC ON")

        # turning on the pulses
        nwa.write("SENS:PULS0 0")
        nwa.write("SENS:PULS1 0")
        nwa.write("SENS:PULS2 0")
        nwa.write("SENS:PULS3 0")
        nwa.write("SENS:PULS4 0")
        # turning off the inverting
        nwa.write("SENS:PULS1:INV 0")
        nwa.write("SENS:PULS2:INV 0")
        nwa.write("SENS:PULS3:INV 0")
        nwa.write("SENS:PULS4:INV 0")

        pnax = nwa
        # hacked to include switches
        # # turning off the pulses
        pnax.write("SENS:PULS0 0")  # automatically sync ADC to pulse gen
        pnax.write("SENS:PULS1 0")
        pnax.write("SENS:PULS2 0")
        pnax.write("SENS:PULS3 0")
        pnax.write("SENS:PULS4 0")
        # turning on the inverting
        pnax.write("SENS:PULS1:INV 1")
        pnax.write("SENS:PULS2:INV 1")
        pnax.write("SENS:PULS3:INV 1")
        pnax.write("SENS:PULS4:INV 1")

        nwa.write("SOUR:POW:COUP OFF")
        nwa.write(":SOURCE:POWER1 %f" % (read_power))
        nwa.write(":SOURCE1:POWER1:MODE ON")
        print ("Read Now On")

        nwa.write(":SOURCE1:POWER3:MODE OFF")
        print ("Qubit Drive Now Off")

        data = nwa.take_one_in_mag_phase()
        fpoints = data[0]
        mags = data[1]
        phases = data[2]
        print ("finished downloading")


        with SlabFile(fname) as f:
            f.append_line('mags', mags)
            f.append_line('phases', phases)
            f.append_line('freq', fpoints)
            f.append_pt('read_power', read_power)

def two_tone_CW(read_freq,probe_freq_center,span,ifbw = ifbw, read_power = read_power,probe_power = probe_power,sweep_pts= sweep_pts,avgs=avgs,avgs_state=avgs_state):
    vary_param = "twotone_CW"

    print("Swept Parameter: %s" % (vary_param))
    prefix = "%s_" % vary_param
    fname = get_next_filename(expt_path, prefix, suffix='.h5')
    print(fname)
    fname = os.path.join(expt_path, fname)

    rfreq =  read_freq

    nwa.set_timeout(10e5)
    nwa.set_ifbw(ifbw)
    nwa.set_sweep_points(sweep_pts)
    nwa.clear_traces()
    nwa.setup_measurement("S21")
    nwa.set_electrical_delay(delay, channel=1)


    nwa.setup_take(averages=avgs)
    nwa.set_average_state(avgs_state)

    probe_start_freq = probe_freq_center-span/2.0
    probe_stop_freq = probe_freq_center+span/2.0
#
    print ("Setting up pulsed parameters")

    nwa.set_ifbw(ifbw)

    nwa.write('SENSE:FOM:STATE 1')
    nwa.write("sense:fom:range2:coupled 0")
    nwa.write("sense:fom:range3:coupled 0")
    nwa.write("sense:fom:range4:coupled 1")
    nwa.write('SENSE:FOM:RANGE1:FREQUENCY:START %f' % probe_start_freq)
    nwa.write('SENSE:FOM:RANGE1:FREQUENCY:STOP %f' % probe_stop_freq)

    nwa.write('SENSE:FOM:RANGE2:FREQUENCY:START %f' % rfreq)
    nwa.write('SENSE:FOM:RANGE2:FREQUENCY:STOP %f' % rfreq)
    nwa.write('SENSE:FOM:RANGE3:FREQUENCY:START %f' % rfreq)
    nwa.write('SENSE:FOM:RANGE3:FREQUENCY:STOP %f' % rfreq)

    print ("set read freq")

    # setting the port powers and decoupling the powers
    nwa.write("SOUR:POW:COUP OFF")

    nwa.write(":SOURCE:POWER1 %f" % (read_power))
    nwa.write(":SOURCE1:POWER1:MODE ON")
    print ("Read Now On")

    nwa.write(":SOURCE:POWER3 %f" % (probe_power))
    nwa.write(":SOURCE1:POWER3:MODE ON")
    print ("Qubit Drive Now ON")

    data = nwa.take_one_in_mag_phase()
    fpoints = data[0]
    mags = data[1]
    phases = data[2]
    print ("finished downloading at frequency %.3f GHz" %(fpoints[0]/10**9))

    with SlabFile(fname) as f:
        f.append_line('freq', fpoints)
        f.append_line('mags', mags)
        f.append_line('phases', phases)
        f.append_pt('read_power', read_power)
        f.append_pt('probe_power', probe_power)
    return fname
def two_tone_power_sweep(probe_power_list,read_freq,probe_freq_center,span,ifbw = ifbw, read_power = read_power,sweep_pts= sweep_pts,avgs=avgs,avgs_state=avgs_state):
    #vary_param = "qubit_twotone_CW"

   # print("Swept Parameter: %s" % (vary_param))
    prefix = "twotoneVsPowerTransmission"
    fname = get_next_filename(expt_path, prefix, suffix='.h5')
    print(fname)
    fname = os.path.join(expt_path, fname)

    rfreq =  read_freq

    nwa.set_timeout(10e5)
    nwa.set_ifbw(ifbw)
    nwa.set_sweep_points(sweep_pts)
    nwa.clear_traces()
    nwa.setup_measurement("S21")
    nwa.set_electrical_delay(delay, channel=1)
    nwa.setup_take(averages=avgs)
    nwa.set_average_state(avgs_state)

    for probe_power in probe_power_list:
        nwa.setup_take(averages=avgs)

        probe_start_freq = probe_freq_center-span/2.0
        probe_stop_freq = probe_freq_center+span/2.0
    #
        print ("Setting up pulsed parameters")

        nwa.set_ifbw(ifbw)

        nwa.write('SENSE:FOM:STATE 1')
        nwa.write("sense:fom:range2:coupled 0")
        nwa.write("sense:fom:range3:coupled 0")
        nwa.write("sense:fom:range4:coupled 1")
        nwa.write('SENSE:FOM:RANGE1:FREQUENCY:START %f' % probe_start_freq)
        nwa.write('SENSE:FOM:RANGE1:FREQUENCY:STOP %f' % probe_stop_freq)

        nwa.write('SENSE:FOM:RANGE2:FREQUENCY:START %f' % rfreq)
        nwa.write('SENSE:FOM:RANGE2:FREQUENCY:STOP %f' % rfreq)
        nwa.write('SENSE:FOM:RANGE3:FREQUENCY:START %f' % rfreq)
        nwa.write('SENSE:FOM:RANGE3:FREQUENCY:STOP %f' % rfreq)

        print ("set read freq")

        # Turning off pulsed aspect of the measurement (just in case it was run before)
        # Turning the leveling of the ports to 'Internal'.   'open-loop' is for pulsed
        nwa.write("source1:power1:alc:mode INTERNAL")
        nwa.write("source1:power3:alc:mode INTERNAL")
        # Setting up the proper trigger mode.   Want to trigger on the point
        nwa.write("TRIG:SOUR EXT")
        nwa.write("SENS:SWE:TRIG:MODE POINT")

        # Turning off the "Reduce IF BW at Low Frequencies" because the PNA-X adjusts the BW automatically to correct for roll-off at low frequencies
        nwa.write("SENS:BWID:TRAC ON")

        # setting the port powers and decoupling the powers
        nwa.write("SOUR:POW:COUP OFF")

        nwa.write(":SOURCE:POWER1 %f" % (read_power))
        nwa.write(":SOURCE1:POWER1:MODE ON")
        print ("Read Now On")

        nwa.write(":SOURCE:POWER3 %f" % (probe_power))
        nwa.write(":SOURCE1:POWER3:MODE ON")
        print ("Qubit Drive Now ON")

        data = nwa.take_one_in_mag_phase()
        fpoints = data[0]
        mags = data[1]
        phases = data[2]

        print ("finished downloading at frequency %.3f GHz" %(fpoints[0]/10**9))

        with SlabFile(fname) as f:
            f.append_line('freq', fpoints)
            f.append_line('mags', mags)
            f.append_line('phases', phases)
            f.append_pt('read_power', read_power)
            f.append_pt('probe_power', probe_power)



# read_power_list = arange(-21,5,1)
# single_tone_power_sweep(read_power_list,read_freq_center = read_freq,span = 100e6)
#
# read_freq  =   7.767947746052817e9
# single_tone_power_sweep(read_power_list,read_freq_center = read_freq,span = 5e6)
#
#
# single_tone_CW(read_freq, 6.17e6, measurement="S21",is_qubitdrive=False,qfreq = 3.810241016669044e9)



# two_tone_CW(read_freq,probe_freq_center=6.6e9, span =  2.4e9)
# two_tone_power_sweep(probe_power_list = arange(-55,0,1),read_freq=read_freq,probe_freq_center=6.0e9,span=2.2e9)

In [ ]:
ts = []
for i in range(50):
    ts.append(time.time())
    print("elapsed time {:0.1f}".format(ts[i]-ts[0]))
#     read_freq  =  7.24e9
    single_tone_CW(read_freq, 1e6,read_power=-40, avgs=1,ifbw = 300,measurement="S21",is_qubitdrive=False,qfreq = 4.865e9)
    single_tone_CW(read_freq, 1e6,read_power=-40, avgs=1,ifbw = 300,measurement="S21",is_qubitdrive=True,qfreq = 4.865e9)
#     two_tone_CW(read_freq,probe_freq_center=4.265e9, span =20e6,read_power=-40,probe_power=-25,sweep_pts=1000)
#     time.sleep(20)


In [ ]:
mags_all=[]
for i in range(27+1,42,2):
    filename = "D:\\OneDriveGershenson-lab\\Rutgers University\\GERSHENSON LAB - Documents\\OurData\\measMW\\T178\\data\\000"+str(i)+"_Readout_single_tone_CW.h5"

    with h5py.File(filename, 'r') as f:
        mags = f['mags']
        freqs = f['freq']
        phases = f['phases']
        read_power=f['read_power']
        mags=np.array(mags[:]).flatten()
        phases=np.array(phases[:]).flatten()
        freqs=np.array(freqs[:]).flatten()
#         for key in f.keys():
#             print(key)
            
    mags_all.append(mags)
ts=np.array(ts)
plt.pcolormesh(freqs,(ts-ts[0])/3600,np.array(mags_all))

plt.show()
# plt.plot(freqs,np.mean(mags_all,axis=0))
# plt.plot(freqs,mags_all[15])
# plt.plot(freqs,mags_all[1])


In [ ]:
filename = "D:\\OneDriveGershenson-lab\\Rutgers University\\GERSHENSON LAB - Documents\\OurData\\measMW\\T172\\data\\scan_53_89.h5"
with SlabFile(filename) as f:
    f.append_line('times', ts)

In [ ]:
import h5py


# h5 = h5py.File(filename,'r')

# print(h5)

# h5.close()

In [ ]:
read_freq=7.238e9
# read_freq=7.192e9

# probe_power=-25
single_tone_CW(read_freq, 200e6,read_power=-40, avgs=10,ifbw = 1000,probe_power=-35,measurement="S21",is_qubitdrive=False,qfreq = 4.44e9)

In [ ]:
read_power_list = arange(-50,0,5)
single_tone_power_sweep(read_power_list,read_freq_center=read_freq, span=5e6, avgs=10,ifbw = 1000,measurement="S21",is_qubitdrive=False,qfreq = 4.44e9)

In [ ]:
filename = "D:\\OneDriveGershenson-lab\\Rutgers University\\GERSHENSON LAB - Documents\\OurData\\measMW\\T187\\data\\00010_Single_tone_nearReadoutFreq_Transmission.h5"

with h5py.File(filename, 'r') as f:
    mags = f['mags']
    freqs = f['freq']
    phases = f['phases']
    read_power=f['read_power']
    read_power=np.array(read_power[:]).flatten()
    mags=np.array(mags[:])
    phases=np.array(phases[:]).flatten()
    freqs=np.array(freqs[0,:]).flatten()
    for key in f.keys():
        print(key)
        
plt.pcolormesh(freqs,read_power,mags)
plt.show()
for i in range(len(read_power)):
    plt.plot(freqs,mags[i])

In [ ]:
    plt.plot(freqs,mags[0])
    plt.plot(freqs,mags[-1])
    print(freqs[np.argmax(mags[0])])

In [ ]:
filename = "D:\\OneDriveGershenson-lab\\Rutgers University\\GERSHENSON LAB - Documents\\OurData\\measMW\\T187\\data\\00004_Readout_single_tone_CW.h5"

with h5py.File(filename, 'r') as f:
    mags = f['mags']
    freqs = f['freq']
    phases = f['phases']
    read_power=f['read_power']
    mags=np.array(mags[:]).flatten()
    phases=np.array(phases[:]).flatten()
    freqs=np.array(freqs[:]).flatten()
    for key in f.keys():
        print(key)
f,ax=plt.subplots()
print(read_power)
ax.plot(freqs,mags,'-')
print(freqs[np.argmax(mags)])

In [ ]:
read_freq=7239501501
# read_freq=7.2385e9
filename=two_tone_CW(read_freq,probe_freq_center=3.91e9, span =100e6,read_power=-40,probe_power=-10,sweep_pts=1000,avgs=5,ifbw=300)
print(filename)
# filename = "D:\\OneDriveGershenson-lab\\Rutgers University\\GERSHENSON LAB - Documents\\OurData\\measMW\\PNAX\\data\\00069_twotone_CW_.h5"
with h5py.File(filename, 'r') as f:
    mags = f['mags']
    freqs = f['freq']
    phases = f['phases']
    mags=np.array(mags[:]).flatten()
    phases=np.array(phases[:]).flatten()
    freqs=np.array(freqs[:]).flatten()
    for key in f.keys():
        print(key)
fig = plt.figure(figsize = (10,5))
ax = fig.add_subplot(111)

ax.plot(freqs,mags,'-')

In [ ]:
plt.plot(freqs,mags,'-')

In [ ]:
from qcodes.instrument.base import Instrument

Instrument.close_all()

In [ ]:
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
LS370 = Model_372(name = 'LS370inputCh', address = 'GPIB::12::INSTR', terminator='\n')
T6 = LS370.ch06.temperature
T6.get()


In [ ]:
LS370.analog_heater.outmode

In [ ]:
LS370.analog_heater.polarity.set('unipolar')

In [ ]:
ts = []
temps=[]
filenames=[]
for i in range(100):
    ts.append(time.time())
#     temps.append(T6.get())
    print("elapsed time {:0.1f}".format(ts[i]-ts[0]))
    filename=two_tone_CW(read_freq,probe_freq_center=3.88e9, span =100e6,read_power=-45,probe_power=-12,sweep_pts=1000,avgs=5,ifbw=300)
    filenames.append(filename)
    time.sleep(1)


In [ ]:
#46 to 107 25 mK

mags_all=[]
t_set=range(46,107,1)
# for i in range(2,8,1):
# for i in range(8,24,1):
for i in range(46,107,1):

    filename = "D:\\OneDriveGershenson-lab\\Rutgers University\\GERSHENSON LAB - Documents\\OurData\\measMW\\T187\\data\\00"+f"{i:03}"+"_twotone_CW_.h5"

    with h5py.File(filename, 'r') as f:
        mags = f['mags']
        freqs = f['freq']
        phases = f['phases']
        read_power=f['read_power']
        mags=np.array(mags[:]).flatten()
        phases=np.array(phases[:]).flatten()
        freqs=np.array(freqs[:]).flatten()
#         for key in f.keys():
#             print(key)
            
    mags_all.append(mags)
# ts=np.array(ts)
plt.pcolormesh(freqs,t_set,np.array(mags_all))
plt.show()

In [ ]:
plt.plot(freqs,np.mean(np.array(mags_all),axis=0))

In [ ]:
mags_all=[]
# for i in range(179,184,1):
# for i in range(184,191,1):
# for i in range(2,8,1):
for i in range(714,735,1):

    filename = "D:\\OneDriveGershenson-lab\\Rutgers University\\GERSHENSON LAB - Documents\\OurData\\measMW\\T187\\data\\00"+f"{i:03}"+"_twotone_CW_.h5"

    with h5py.File(filename, 'r') as f:
        mags = f['mags']
        freqs = f['freq']
        phases = f['phases']
        read_power=f['read_power']
        mags=np.array(mags[:]).flatten()
        phases=np.array(phases[:]).flatten()
        freqs=np.array(freqs[:]).flatten()
#         for key in f.keys():
#             print(key)
            
    mags_all.append(mags)
ts=np.array(ts)
print(ts)
plt.pcolormesh(freqs,ts,np.array(mags_all))
plt.show()

In [ ]:
plt.plot(freqs,np.mean(np.array(mags_all),axis=0))

In [ ]:
mags_all=[]
# for i in range(179,184,1):
# for i in range(184,191,1):
# for i in range(2,8,1):
for i in range(143,643,1):

    filename = "D:\\OneDriveGershenson-lab\\Rutgers University\\GERSHENSON LAB - Documents\\OurData\\measMW\\T187\\data\\00"+f"{i:03}"+"_twotone_CW_.h5"

    with h5py.File(filename, 'r') as f:
        mags = f['mags']
        freqs = f['freq']
        phases = f['phases']
        read_power=f['read_power']
        mags=np.array(mags[:]).flatten()
        phases=np.array(phases[:]).flatten()
        freqs=np.array(freqs[:]).flatten()
#         for key in f.keys():
#             print(key)
            
    mags_all.append(mags)
ts=np.array(ts)
print(ts)
plt.pcolormesh(freqs,ts,np.array(mags_all))
plt.show()


In [ ]:
plt.plot(freqs,np.mean(np.array(mags_all),axis=0))

In [ ]:
two_tone_power_sweep(probe_power_list=np.arange(-10,10,2),read_freq=read_freq,probe_freq_center=5.72e9, span =100e6,read_power=-40,sweep_pts=1000,avgs=10,ifbw=300)

In [ ]:
np.arange(-20,-5,2)

In [ ]:
filename = "D:\\OneDriveGershenson-lab\\Rutgers University\\GERSHENSON LAB - Documents\\OurData\\measMW\\T182\\data\\00003_twotoneVsPowerTransmission.h5"

with h5py.File(filename, 'r') as f:
    mags = f['mags']
    freqs = f['freq']
    phases = f['phases']
    probe_power=f['probe_power']
    probe_power=np.array(probe_power[:]).flatten()
    mags=np.array(mags[:])
    phases=np.array(phases[:]).flatten()
    freqs=np.array(freqs[0,:]).flatten()
    for key in f.keys():
        print(key)

In [ ]:
plt.pcolormesh(freqs,probe_power,mags)

In [ ]:
avec=probe_power
fvec=freqs
mag=10**(mags/20)
Qfaclst=[]
linewidth=[]
reslst=[]
f,ax=plt.subplots()
for i in range(len(avec)):
    p = fitlor(fvec,-mag[i],showfit=False)
    ax.plot(fvec,-lorfunc(p,fvec),'r-',alpha=1)
    ax.plot(fvec,mag[i])

    nu_res = p[2]
    Qfac = nu_res/p[3]/2
    Qfaclst=np.append(Qfaclst,Qfac)
    linewidth=np.append(linewidth,p[3])
    reslst=np.append(reslst,nu_res)
    

#plt.xlim(149.6e6,150e6)

In [ ]:
f,ax=plt.subplots()

ax.plot(avec[1:],reslst[1:],'x')
print('dispersion= ',np.max(reslst)-np.min(reslst[1:]))

In [ ]:
f,ax=plt.subplots()

#ax.plot(avec,Qfaclst,'x')
ax.plot(avec[1:],2*linewidth[1:]/1e6,'x')

# ax.set_xscale('log')
# ax.set_yscale('log')

ax.set_xlabel('amplitude of 2nd tone (14dBm)',size=16)
ax.set_ylabel('line width (MHz)',size=16)

In [ ]:
from slab.dsfit import*

In [ ]:
filename = "D:\\OneDriveGershenson-lab\\Rutgers University\\GERSHENSON LAB - Documents\\OurData\\measMW\\PNAX\\data\\00064_twotone_CW_.h5"

with h5py.File(filename, 'r') as f:
    mags = f['mags']
    freqs = f['freq']
    phases = f['phases']
    mags=np.array(mags[:]).flatten()
    phases=np.array(phases[:]).flatten()
    freqs=np.array(freqs[:]).flatten()
    for key in f.keys():
        print(key)

In [ ]:
fig = plt.figure(figsize = (10,5))
ax = fig.add_subplot(111)

ax.plot(freqs,mags,'-')

In [ ]:
fig = plt.figure(figsize = (10,5))
ax = fig.add_subplot(111)

ax.plot(freqs,mags,'-')

In [ ]:
fig = plt.figure(figsize = (10,5))
ax = fig.add_subplot(111)

ax.plot(freqs,mags,'-')

In [ ]:
from slab.dsfit import *

In [ ]:
f,ax=plt.subplots()
p = fitlor(freqs,10**(-mags/np.max(mags)),showfit=True)
# ax.plot(freqs,-lorfunc(p,freqs),'r-')
# ax.plot(freqs,-20**(-mags/np.max(mags)))

nu_res = p[2]
Qfac = nu_res/p[3]/2
print(Qfac)

In [ ]:
plt.plot(20**(-mags/np.max(mags)),np.abs(phases))

In [ ]:
nu_res

In [ ]:
Qfac

In [ ]:
np.max(mags)